In [182]:
import os 
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import re
import pytesseract
import pandas as pd


pathname0 = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/Lines0'
pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/Lines'


def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def listdir_nohidden1(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
class Convert_to_line:
    def __init__(self,im):
        self.im=im
    
    def get_concat_h(self,im1, im2):
        dst = Image.new('RGB', (im1.width + im2.width, im1.height))
        dst.paste(im1, (0, 0))
        dst.paste(im2, (im1.width, 0))
        return dst

    def crop(self):
        #Top Crop
        width,height=self.im.size
        top=0.3*height
        bottom=0.7*height

        left=0.1*width
        right=1*width
        IM1=self.im.crop((left,top,right,bottom))
        return IM1

def cjk_detect(texts):
    # korean
    if re.search("[\uac00-\ud7a3]", texts):
        return "ko"
    # japanese
    if re.search("[\u3040-\u30ff]", texts):
        return "ja"
    # chinese
    if re.search("[\u4e00-\u9FFF]", texts):
        return "zh"
    return None

pattern = re.compile(
    "\u3041-\u3096" # Hiragana
    "\u30A0-\u30FF" # Katakana
)


def Count(text):
    i=0
    num=0
    text = text.replace('\n',"")
    for row in text:
        if "子" in row:
            i=i+1        
            num=num+1
        if cjk_detect(row) == "ja":
            if cjk_detect(text[i+1]) == "ja":
                i=i+1
                num=num+1
            else:
                i=i+1
    return(num)

In [2]:
#Step 1: Convert pdf to png
import glob, sys, fitz

# To get better resolution
zoom_x = 2.0  # horizontal zoom
zoom_y = 2.0  # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension

all_files = glob.glob(r"C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/All.pdf")

for filename in all_files:
    doc = fitz.open(filename)  # open document
    for page in doc:  # iterate through the pages
        if len(str(page.number))==1:
            print("")
        if len(str(page.number))==2:
            pix = page.get_pixmap(matrix=mat)  # render page to an image
            pix.save(r"C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/Lines0/0%i-page.png" % page.number)  # store image as a PNG
        else:
            pix = page.get_pixmap(matrix=mat)  # render page to an image
            pix.save(r"C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/Lines0/%i-page.png" % page.number)  # store image as a PNG

In [183]:
#Step2
#Convert to line
from PIL import Image
import os, cv2

def listdir_nohidden1(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
            
for i in range(0,len(sorted(listdir_nohidden1(pathname0)))):
    image = sorted(listdir_nohidden1(pathname0))[i]
    image_test = Image.open(pathname0+"/"+image)
    
    test=Convert_to_line(image_test)
    line=test.crop() #Remove redundant space around image
    
    line.save(pathname+"/"+image)

In [184]:
#Step3 Combine to single line
allfiles = os.listdir(pathname)
imlist =[cv2.imread(pathname+"/"+filename) for filename in np.flip(allfiles) if filename[-4:] in [".png"]]
im = np.hstack(imlist)

In [199]:
#Detecting circles
cropped_image = im[0:50, 0:im.shape[1]]

# Detecting circles: This step is heavily refering to the following website https://www.geeksforgeeks.org/circle-detection-using-opencv-python/
# Convert to grayscale.
gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
# Blur using 3 * 3 kernel.
gray_blurred = cv2.blur(gray, (3, 3))  

# Apply Hough transform on the blurred image.
detected_circles = cv2.HoughCircles(gray_blurred, 
                   cv2.HOUGH_GRADIENT, 1, 150, param1 = 50,
               param2 = 15, minRadius = 8, maxRadius = 12)
detected_circles = np.uint16(np.around(detected_circles))

In [200]:
#Crop into departments
circle_list=sorted(detected_circles[0], key=lambda x: x[0])

pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/Departments'

for i in range(0,len(circle_list)+1):
    try:        
        if i ==0:
            dept = im[0:im.shape[0], 0:circle_list[0][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
            i=i+1
        if i>0 and len(str(i))==1:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
            i=i+1
        if i>0 and len(str(i))==2:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept0"+str(i)+".png")
            i=i+1
        if i ==len(detected_circles1[0])+1:
            dept = im[0:im.shape[0], circle_list[i][0]+15:im.shape[1]]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")
            i=i+1
        else:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")
            i=i+1
    except:
        i=i+1
    


In [206]:
#Erase miss-detected departments
indexes = [4,7,8,11,12,13,15,16,17,18,19,21,22,23,24,26,27,29,32,33,34,35,36,37,38,39,41,42,43,44,45,46,48,49,52,54,55,56,57,58,59,60,61,62,63,64,65,67,68,69,70,71,73,75]
circle_list = [circle_list[i] for i in indexes]

for i in range(0,len(circle_list)+1):
    try:        
        if i ==0:
            dept = im[0:im.shape[0], 0:circle_list[0][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
            i=i+1
        if i>0 and len(str(i))==1:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
            i=i+1
        if i>0 and len(str(i))==2:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept0"+str(i)+".png")
            i=i+1
        if i ==len(detected_circles1[0])+1:
            dept = im[0:im.shape[0], circle_list[i][0]+15:im.shape[1]]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")
            i=i+1
        else:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")
            i=i+1
    except:
        i=i+1


In [14]:
#Extracting department names
pathname=r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1937/Departments'
allfiles = os.listdir(pathname)
for i in range(0,len(allfiles)):    
    imlist =[cv2.imread(pathname+"/"+filename) for filename in np.flip(allfiles) if filename[-4:] in [".png"]]
    im = np.hstack(imlist)
    Image.fromarray(im).save(pathname+'/Aggregate.png')

107

"Aggregate.png" must be scanned by Clova to extract out text.

In [209]:
#Counting female names
pathname=r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1935/Departments'
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

dict = {'Size':[],
        'Female':[]}
df = pd.DataFrame(dict)

for i in range(2,len(sorted(listdir_nohidden1(pathname)))-1):
    im=sorted(listdir_nohidden1(pathname))[i]
    image=cv2.imread(pathname+"/"+im)
    img = Image.fromarray(image)
    text = pytesseract.image_to_string(img,lang='jpn_vert')
    df2={'Size':image.shape[1], 'Female': Count(text)}
    df=df.append(df2, ignore_index = True)

df["Year"]=1935


C:\Temp\ipykernel_7200\2004834288.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df2, ignore_index = True)
C:\Temp\ipykernel_7200\2004834288.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df2, ignore_index = True)
C:\Temp\ipykernel_7200\2004834288.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df2, ignore_index = True)
C:\Temp\ipykernel_7200\2004834288.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df2, ignore_index = True)
C:\Temp\ipykernel_7200\2004834288.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

AttributeError: 'NoneType' object has no attribute '__array_interface__'

In [212]:
#Merge with name data
df1=pd.read_csv(pathname+"/Aggregate.csv", header=None)
df1=df1.rename(columns={0: 'Dept_name', 1: 'none'})
df=pd.concat([df, df1], axis=1)
df.to_csv(pathname+"/Data.csv")